Analyse of bikeshare.metro.net

In [7]:
active_vehicle= stationData[stationData['Status'] == 'Active']
inactive_vehicle = stationData[stationData['Status'] == 'Inactive']

# Count the number of active vehicles   
num= active_vehicle['Status'].count()
print(f'The number of active vehicles is: {num}')

active_vehicle


NameError: name 'stationData' is not defined

In [ ]:
stationData.isna().sum()

Kiosk ID        0
Kiosk Name      0
Go Live Date    0
Region          2
Status          0
Latitude        0
Longitude       0
status2         0
dtype: int64

Active Vehicles grouped by Region


In [ ]:
active_vehicle.columns = active_vehicle.columns.str.strip()
counts = (
    active_vehicle
      .groupby('Region')
      .size()
      .reset_index(name='active_count')
)
counts


,Region,active_count
0,DTLA,143
1,North Hollywood,12
2,Westside,64


In [ ]:
inactive_vehicle.columns = inactive_vehicle.columns.str.strip()
counts = (
    inactive_vehicle
      .groupby('Region')
      .size()
      .reset_index(name='inactive_count')
)
counts

,Region,inactive_count
0,DTLA,59
1,Free Bikes,2
2,North Hollywood,24
3,Pasadena,31
4,Port of LA,13
5,Westside,79


Verschiedene Regionen

In [ ]:
regions = (
    inactive_vehicle
      .groupby('Region')
      .size()
      .reset_index(name='inactive_count')
)
regions

,Region,inactive_count
0,DTLA,59
1,Free Bikes,2
2,North Hollywood,24
3,Pasadena,31
4,Port of LA,13
5,Westside,79


Trip Data

In [ ]:
tripData = pd.read_csv(
    'data/metro-trips-2025-q1.csv',
    encoding='latin-1'
)

tripData.columns=tripData.columns.str.strip()

tripData

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,475609834,5,1/1/2025 0:12,1/1/2025 0:17,3030,34.051941,-118.243530,4491,34.047440,-118.247940,13668,30,One Way,Monthly Pass,standard
1,475609846,7,1/1/2025 0:12,1/1/2025 0:19,4558,34.025688,-118.395302,4569,34.026550,-118.408463,30021,30,One Way,Monthly Pass,electric
2,475609903,11,1/1/2025 0:13,1/1/2025 0:24,4212,33.988129,-118.471741,4206,33.998341,-118.461014,14923,30,One Way,Monthly Pass,standard
3,475609904,11,1/1/2025 0:13,1/1/2025 0:24,4212,33.988129,-118.471741,4206,33.998341,-118.461014,26704,30,One Way,Monthly Pass,electric
4,475610048,13,1/1/2025 0:27,1/1/2025 0:40,4472,34.092602,-118.280930,4509,34.101639,-118.309174,14790,30,One Way,Monthly Pass,standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95911,497792778,13,3/31/2025 23:43,3/31/2025 23:56,3022,34.046070,-118.233093,4680,34.043522,-118.255089,13669,30,One Way,Monthly Pass,standard
95912,497792596,7,3/31/2025 23:44,3/31/2025 23:51,4643,34.072620,-118.449440,4528,34.060970,-118.444366,25531,30,One Way,Monthly Pass,electric
95913,497792744,6,3/31/2025 23:49,3/31/2025 23:55,3026,34.063179,-118.245880,4445,34.073639,-118.251572,29109,30,One Way,Monthly Pass,electric
95914,497826163,623,3/31/2025 23:56,4/1/2025 10:19,3035,34.048401,-118.260948,3077,34.039871,-118.250038,5755,365,One Way,Annual Pass,standard


In [ ]:
tripData.isna().sum()

trip_id                   0
duration                  0
start_time                0
end_time                  0
start_station             0
start_lat                19
start_lon                19
end_station               0
end_lat                2506
end_lon                2506
bike_id                   0
plan_duration             0
trip_route_category       0
passholder_type           0
bike_type                 0
dtype: int64

In [ ]:
meantime=tripData['duration'].mean()
meantime

np.float64(29.596250886192085)

In [ ]:
import datetime

# Get the weekday number and name for a specific date, maybe needed for analysis for tripdata

d = datetime.date(2025, 5, 25)
nummer = d.weekday()
name_en = d.strftime("%A")
print(f'The number of weekday is: {nummer}')
print(f'The name of the weekday is: {name_en}')

tripData["start_time"]


The number of weekday is: 6
The name of the weekday is: Sunday


0          1/1/2025 0:12
1          1/1/2025 0:12
2          1/1/2025 0:13
3          1/1/2025 0:13
4          1/1/2025 0:27
              ...       
95911    3/31/2025 23:43
95912    3/31/2025 23:44
95913    3/31/2025 23:49
95914    3/31/2025 23:56
95915    3/31/2025 23:59
Name: start_time, Length: 95916, dtype: object

In [ ]:
df= tripData.copy()

# 1. Parse mit explizitem Format
df['start_time'] = pd.to_datetime(
    df['start_time'],
    format='%m/%d/%Y %H:%M'   # Month/Day/Year Hour:Minute
)

# 2. Englischen Wochentag als neue Spalte
df['weekday'] = df['start_time'].dt.day_name()

# 3. Funktion zur Kategorisierung der Tageszeit
def get_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# 4. Tageszeit-Spalte
df['time_of_day'] = df['start_time'].dt.hour.apply(get_time_of_day)

print(df[['start_time', 'weekday', 'time_of_day']])

               start_time    weekday time_of_day
0     2025-01-01 00:12:00  Wednesday       Night
1     2025-01-01 00:12:00  Wednesday       Night
2     2025-01-01 00:13:00  Wednesday       Night
3     2025-01-01 00:13:00  Wednesday       Night
4     2025-01-01 00:27:00  Wednesday       Night
...                   ...        ...         ...
95911 2025-03-31 23:43:00     Monday       Night
95912 2025-03-31 23:44:00     Monday       Night
95913 2025-03-31 23:49:00     Monday       Night
95914 2025-03-31 23:56:00     Monday       Night
95915 2025-03-31 23:59:00     Monday       Night

[95916 rows x 3 columns]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95916 entries, 0 to 95915
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   trip_id              95916 non-null  int64         
 1   duration             95916 non-null  int64         
 2   start_time           95916 non-null  datetime64[ns]
 3   end_time             95916 non-null  object        
 4   start_station        95916 non-null  int64         
 5   start_lat            95897 non-null  float64       
 6   start_lon            95897 non-null  float64       
 7   end_station          95916 non-null  int64         
 8   end_lat              93410 non-null  float64       
 9   end_lon              93410 non-null  float64       
 10  bike_id              95916 non-null  int64         
 11  plan_duration        95916 non-null  int64         
 12  trip_route_category  95916 non-null  object        
 13  passholder_type      95916 non-

In [ ]:
statisticWeek = (
    df
    .groupby('weekday')
    .agg(
        trip_count=('trip_id', 'size'),
        avg_duration=('duration', 'mean')
    )
    .reset_index()
)

statisticWeek 

,weekday,trip_count,avg_duration
0,Friday,13799,26.876440
1,Monday,13829,26.281582
2,Saturday,15243,36.570885
3,Sunday,14154,40.866469
4,Thursday,12665,25.835057
5,Tuesday,13525,23.494122
6,Wednesday,12701,25.478702


In [ ]:
statisticTime = (
    df
    .groupby('time_of_day')
    .agg(
        trip_count=('trip_id', 'size'),
        avg_duration=('duration', 'mean')
    )
    .reset_index()
)
statisticTime 

,time_of_day,trip_count,avg_duration
0,Afternoon,39843,33.107798
1,Evening,25705,26.916009
2,Morning,21653,26.834249
3,Night,8715,28.310040


In [ ]:
num_df = df.select_dtypes(include='number')

# 3. Berechnung der Korrelationsmatrix
corr_matrix = num_df.corr()

print(corr_matrix)

                trip_id  duration  start_station  start_lat  start_lon  \
trip_id        1.000000  0.005852       0.040216   0.001681  -0.041365   
duration       0.005852  1.000000       0.042648  -0.095278  -0.092962   
start_station  0.040216  0.042648       1.000000  -0.099173  -0.587270   
start_lat      0.001681 -0.095278      -0.099173   1.000000   0.470076   
start_lon     -0.041365 -0.092962      -0.587270   0.470076   1.000000   
end_station    0.044810  0.021577       0.624296  -0.097367  -0.553514   
end_lat        0.000440 -0.115427      -0.112841   0.916713   0.471255   
end_lon       -0.042445 -0.110882      -0.576726   0.466042   0.980325   
bike_id       -0.019890 -0.010696      -0.073228   0.083791   0.091659   
plan_duration -0.005150 -0.082531       0.014829   0.104399   0.063829   

               end_station   end_lat   end_lon   bike_id  plan_duration  
trip_id           0.044810  0.000440 -0.042445 -0.019890      -0.005150  
duration          0.021577 -0.115427 

In [ ]:
num_df = stationData.select_dtypes(include='number')

# 3. Berechnung der Korrelationsmatrix
corr_matrix = num_df.corr()

print(corr_matrix)

           Kiosk ID  Latitude  Longitude
Kiosk ID   1.000000  0.060770  -0.011444
Latitude   0.060770  1.000000  -0.651801
Longitude -0.011444 -0.651801   1.000000


1. Infos about the data quality

In [ ]:
tripInfo = tripData.info()
print(tripInfo)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95916 entries, 0 to 95915
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   trip_id              95916 non-null  int64  
 1   duration             95916 non-null  int64  
 2   start_time           95916 non-null  object 
 3   end_time             95916 non-null  object 
 4   start_station        95916 non-null  int64  
 5   start_lat            95897 non-null  float64
 6   start_lon            95897 non-null  float64
 7   end_station          95916 non-null  int64  
 8   end_lat              93410 non-null  float64
 9   end_lon              93410 non-null  float64
 10  bike_id              95916 non-null  int64  
 11  plan_duration        95916 non-null  int64  
 12  trip_route_category  95916 non-null  object 
 13  passholder_type      95916 non-null  object 
 14  bike_type            95916 non-null  object 
dtypes: float64(4), int64(6), object(5)
m

Procedure: Corrupt data:

Needed collums: duration, start_time, end_time, start_station, start_lat, start_lon , end_station , end_lat, end_lon, bike_type

1. trip_id ( low possibility )
    if NaN then delet complete row
2. duration(its a derived value, its computable)  (low possibility)
    If NaN take calc with start and end time, or take avg of same start and end station
3. start_time/end_time: 
    calc with duration and start/end time



In [ ]:
stationInfo = stationData.info()
print(stationInfo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Kiosk ID      429 non-null    int64  
 1   Kiosk Name    429 non-null    object 
 2   Go Live Date  429 non-null    object 
 3   Region        427 non-null    object 
 4   Status        429 non-null    object 
 5   Latitude      429 non-null    float64
 6   Longitude     429 non-null    float64
 7   status2       429 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 26.9+ KB
None
